In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import hist

plt.style.use(hep.style.CMS)

In [ ]:
def add_uncertainty(hist, ax, newaxis, ratio=False):
    opts = {
        "step": "post",
        "label": "Uncertainty",
        "hatch": "///",
        "facecolor": "none",
        "edgecolor": (0, 0, 0, 0.5),
        "linewidth": 0,
        "zorder": 10.0,
    }

    if ratio:
        down = np.ones(len(hist.counts())) - hist.errors() / hist.counts()
        up = np.ones(len(hist.counts())) + hist.errors() / hist.counts()
    else:
        down = hist.counts() - hist.errors()
        up = hist.counts() + hist.errors()
    up = np.nan_to_num(up)[: len(newaxis) - 1]
    down = np.nan_to_num(down)[: len(newaxis) - 1]
    ax.fill_between(x=newaxis, y1=np.r_[down, down[-1]], y2=np.r_[up, up[-1]], **opts)

In [ ]:
fitDiagnostics = uproot.open("Fitting/fitDiagnostics.prefit-asimov.root")

In [ ]:
shapetype = "shapes_prefit"
fitDiagnostics[shapetype].keys()

In [ ]:
processes = [
    "ttgamma",
    "NonPrompt",
    "MisID",
    "WG",
    "ZG",
    "other",
]

processes.reverse()

colors = {
    "ttgamma": "",  # FIXME
}

labels = {
    "ttgamma": r"$t\bar{t}\gamma$",
    "NonPrompt": r"nonprompt $\gamma$",
    "MisID": r"MisID $\gamma$",
    "WG": r"$W\gamma$",
    "ZG": r"$Z\gamma$",
    "other": r"other",
}

In [ ]:
groupingCategory = {
    "NonPrompt": [3j, 4j],
    "MisID": [2j],
    "Prompt": [1j],
}

groupingFineCategory = {
    "Genuine Photon": [1j],
    "Misidentified Electron": [2j],
    "Hadronic Photon": [3j],
    "Hadronic Fake": [4j],
}

groupingMCDatasets = {
    "ZG": [
        "ZGamma_01J_5f_lowMass",
    ],
    "WG": [
        "WGamma",
    ],
    "other": [
        "TTbarPowheg_Dilepton",
        "TTbarPowheg_Semilept",
        "TTbarPowheg_Hadronic",
        "W2jets",
        "W3jets",
        "W4jets",
        "DYjetsM50",
        "ST_s_channel",
        "ST_tW_channel",
        "ST_tbarW_channel",
        "ST_tbar_channel",
        "ST_t_channel",
        "TTWtoLNu",
        "TTWtoQQ",
        "TTZtoLL",
        "GJets_HT200To400",
        "GJets_HT400To600",
        "GJets_HT600ToInf",
        "ZZ",
        "WZ",
        "WW",
        "TGJets",
    ],
    "ttgamma": [
        "TTGamma_Dilepton",
        "TTGamma_SingleLept",
        "TTGamma_Hadronic",
    ],
}

In [ ]:
# axes hardcoded from save_to_root.py
inputshapes = {
    "m3": "RootFiles/M3_Output.root",
    "misidEle": "RootFiles/MisID_Output_electron.root",
    "Vgamma": "RootFiles/MisID_Output_muon.root",
    "chIso": "RootFiles/Isolation_Output.root",
}
newaxes = {
    channel: uproot.open(fn)["data_obs"].axes[0].edges()
    for channel, fn in inputshapes.items()
}
axeslabels = {
    "m3": r"$M_3$ [GeV]",
    "misidEle": r"$m_{e\gamma}$ [GeV]",
    "Vgamma": r"$m_{\mu\gamma}$ [GeV]",
    "chIso": "Charged Hadron Isolation [GeV]",
}

In [ ]:
channel = "Vgamma"
dc_x = fitDiagnostics[f"{shapetype}/{channel}"]
h = dc_x["data"]

fig, (ax, rax) = plt.subplots(
    2,
    1,
    figsize=(10, 10),
    gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05},
    sharex=True,
)
newaxis = newaxes[channel]
xlabel = axeslabels[channel]

hep.cms.label(
    "Preliminary",
    data=True,
    lumi=35.9,
    loc=0,
    ax=ax,
)

hep.histplot(
    [dc_x[x].counts()[: len(newaxis) - 1] for x in processes],
    bins=newaxis,
    # w2=[ dc_sr[x].errors() for x in processes ],  # not needed
    histtype="fill",
    stack=True,
    label=[labels[x] for x in processes],
    # color=['red', 'green'],
    ax=ax,
)

# this is for the data points
wq = hep.histplot(
    dc_x["data"].values()[1][: len(newaxis) - 1],
    bins=newaxis,
    w2=dc_x["data"].values()[1][: len(newaxis) - 1],
    histtype="errorbar",
    stack=False,
    label="Observation",
    color="black",
    ax=ax,
)

ratio_val = dc_x["data"].values()[1] / dc_x["total"].counts()
ratio_err_hep = np.sqrt(dc_x["data"].values()[1]) / dc_x["total"].counts()

# this is for the ratio plot
hep.histplot(
    ratio_val[: len(newaxis) - 1],
    bins=newaxis,
    yerr=ratio_err_hep[: len(newaxis) - 1],
    histtype="errorbar",
    color="black",
    ax=rax,
)

rax.set_ylim(0, 1.99)
rax.set_xlabel(xlabel)
rax.set_ylabel(r"Data/Pred.")
ax.set_ylabel(r"Events")

add_uncertainty(dc_x["total"], ax, newaxis)
add_uncertainty(dc_x["total"], rax, newaxis, ratio=True)

# ax.legend()
ax.legend(prop={"size": 15}, bbox_to_anchor=(1, 1))